In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
from scipy.spatial.distance import cdist

In [2]:
csv = pd.read_csv('Data-Crawl-Google\multi_page_data_crawled.csv')
csv.head()

<>:1: SyntaxWarning: invalid escape sequence '\m'
<>:1: SyntaxWarning: invalid escape sequence '\m'
C:\Users\ASUS\AppData\Local\Temp\ipykernel_24076\3633698349.py:1: SyntaxWarning: invalid escape sequence '\m'
  csv = pd.read_csv('Data-Crawl-Google\multi_page_data_crawled.csv')


,Unnamed: 0,judul berita,penerbit,sinopsis,link
0,0,Indonesia tunjukkan transformasi pendidikan le...,ANTARA News Jambi,ANTARA - Indonesia menunjukkan sejumlah platfo...,https://jambi.antaranews.com/rilis-pers/436953...
1,1,Kemendikbudristek Temukan 4 Kecurangan Penerim...,detikcom,Balai Pembiayaan Pendidikan Tinggi (BPPT) Keme...,https://www.detik.com/edu/beasiswa/d-7564848/k...
2,2,Kemendikbud Ungkap Pelanggaran Mahasiswa Pener...,Kompas.com,Kemendikbud Ristek menungkap beragam pelanggar...,https://www.kompas.com/edu/read/2024/10/01/165...
3,3,Staf Khusus Nadiem Beberkan Tiga Tantangan Bes...,Medcom.id,Bali: Staf Khusus Mendikbudristek Bidang Kompe...,https://www.medcom.id/pendidikan/news-pendidik...
4,4,Pemuda Didorong Berperan Aktif dalam Kebijakan...,Borneonews,Billy Mambrasar mengajak pemuda berperan aktif...,https://www.borneonews.co.id/berita/385694-pem...


In [4]:
# get the text only and convert to numpy array
text = csv['judul berita'].to_numpy()
np.size(text)

191

In [5]:
vectorizer = TfidfVectorizer(use_idf=False, norm= None, ngram_range=(1,2))
matrix = vectorizer.fit_transform(text).toarray()

In [6]:
cluster_numb = 2
model = KMeans(n_clusters=cluster_numb, max_iter=100000, n_init=1)
model.fit(matrix)

KMeans(max_iter=100000, n_clusters=2, n_init=1)

In [7]:
print("Top terms per cluster:")
order_centroids = model.cluster_centers_.argsort()[:,::-1]
terms = vectorizer.get_feature_names_out()
for i in range(cluster_numb):
    print("\n----------Cluster {}----------\n".format(i))
    for ind in order_centroids[i, :10]:
        print(terms[ind] + " - " + str(ind))

Top terms per cluster:

----------Cluster 0----------

nusantara berita - 1395
pendidikan indonesia - 1514
upi - 2182
nusantara - 1394
se nusantara - 1812
se - 1811
titik se - 2094
titik - 2093
lebih - 1151
pohon lebih - 1649

----------Cluster 1----------

pendidikan - 1496
indonesia - 750
di - 478
pendidikan indonesia - 1514
dan - 382
di indonesia - 494
dalam - 359
2024 - 13
pendidikan di - 1508
untuk - 2157


In [8]:
#this part is problematic
vectorizer.fit(text)
final_sentiment = []

for i in text:
    # print(i)
    y = vectorizer.transform([i])
    y = model.predict(y)
    # print(np.shape(y))
    final_sentiment.append(int(y[0]))

In [13]:
csv['cluster'] = final_sentiment
csv

,Unnamed: 0,judul berita,penerbit,sinopsis,link,sentiment,cluster
0,0,Indonesia tunjukkan transformasi pendidikan le...,ANTARA News Jambi,ANTARA - Indonesia menunjukkan sejumlah platfo...,https://jambi.antaranews.com/rilis-pers/436953...,1,1
1,1,Kemendikbudristek Temukan 4 Kecurangan Penerim...,detikcom,Balai Pembiayaan Pendidikan Tinggi (BPPT) Keme...,https://www.detik.com/edu/beasiswa/d-7564848/k...,1,1
2,2,Kemendikbud Ungkap Pelanggaran Mahasiswa Pener...,Kompas.com,Kemendikbud Ristek menungkap beragam pelanggar...,https://www.kompas.com/edu/read/2024/10/01/165...,1,1
3,3,Staf Khusus Nadiem Beberkan Tiga Tantangan Bes...,Medcom.id,Bali: Staf Khusus Mendikbudristek Bidang Kompe...,https://www.medcom.id/pendidikan/news-pendidik...,1,1
4,4,Pemuda Didorong Berperan Aktif dalam Kebijakan...,Borneonews,Billy Mambrasar mengajak pemuda berperan aktif...,https://www.borneonews.co.id/berita/385694-pem...,1,1
...,...,...,...,...,...,...,...
186,186,Sisi Gelap Transformasi Pendidikan Indonesia d...,Pemerintah Kabupaten Pemalang,Mendikbud Sebut Kartu Indonesia Pintar Upaya T...,https://pemalangkab.go.id/2020/02/pendidikan-k...,1,1
187,187,Mendikbud Sebut Kartu Indonesia Pintar Upaya T...,detikNews,KEMENKO PMK — Indonesia mulai menjalin kerja s...,https://news.detik.com/berita/d-7472320/hashim...,1,1
188,188,"Demi Wujudkan SDM Unggul, Pemerintah Indonesia...",Sekretariat Kabinet Republik Indonesia,"Sesuai dengan pidato Rektor dalam upacara, Uni...",https://setkab.go.id/hardiknas-2024-seskab-dor...,1,1
189,189,"Bergerak Bersama, Majukan Pendidikan Indonesia",Medcom.id,Meskipun lulusan perguruan tinggi bukan menjad...,https://www.medcom.id/pendidikan/news-pendidik...,1,1


In [14]:
for i in range(191):
    if csv['cluster'][i] == 0:
        print(csv['judul berita'][i])

Paguyuban KSE Universitas Pendidikan Indonesia Berperan dalam Gerakan Tanam Pohon: Lebih dari 4.000 Bubut Ditanam di 40 Titik Se-Nusantara – BERITA UPI
